In [1]:
from os import environ
environ.setdefault("HF_HOME", "/data/hf_models/")

'/data/hf_models/'

In [2]:
import sys
sys.path.append("../../src/")

In [ ]:
from agent_design_pattern.agent import AgentMessage, BaseAgent
from agent_design_pattern.orchestration import LoopAgent
from chain import CasualSingleTurnChain

In [4]:
class Agent(BaseAgent):
    def __init__(self, chain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.chain = chain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.name
        return message

In [5]:
from typing import Generator


system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

def iterator(max_iteration: int) -> Generator[bool, None, None]:
    n = 0
    while True:
        yield n >= max_iteration
        n = (n + 1) % (max_iteration + 1)

chain = CasualSingleTurnChain(model="ibm-granite/granite-4.0-h-1b", system_prompt=system_prompt, user_prompt_template=user_prompt)
agent = Agent(chain, state_change_callback=state_callback)
loop_agent = LoopAgent(agent, iterator(max_iteration), state_change_callback=state_callback)

The fast path is not available because one of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [ ]:
query = "Write a short poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=-1, temperature=1.2, max_new_tokens=16384)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: running iteration 3
agent state: running iteration 4
agent state: idle
query="Write a short poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response='Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA' responses=[('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA')] context=None execution_result='success' error_message=None
message respo

In [ ]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=1, temperature=1.5, max_new_tokens=16384)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: running iteration 3
agent state: running iteration 4
agent state: idle
query="Write a poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response='Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA' responses=[('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA')] context=None execution_result='success' error_message=None
message responses len: 1
Agent_1: Upon the tranquil sea, a calm repose,
Where gentle breezes softly kiss the shore,
A
--------------------------------------------------



In [ ]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=3, temperature=1.5, max_new_tokens=16384)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: running iteration 3
agent state: running iteration 4
agent state: idle
query="Write a poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response='Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA' responses=[('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA')] context=None execution_result='success' error_message=None
message responses len: 3
Agent_1: Upon the tranquil sea, a calm repose,
Where gentle breezes softly kiss the shore,
A
--------------------------------------------------

Agent_1: Upon the tranquil sea, a calm repose,
Where 

In [9]:
from typing import List


def custom_keep_logic(agent_name: str, message: AgentMessage, response: List[str]) -> List[str]:
    return response


query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=custom_keep_logic, temperature=1.5, max_new_tokens=16384)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: running iteration 3
agent state: running iteration 4
agent state: idle
query="Write a poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response='Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA' responses=[('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA'), ('Agent_1', 'Upon the tranquil sea, a calm repose,\nWhere gentle breezes softly kiss the shore,\nA')] context=None execution_result='success' error_message=None
message responses l